In [10]:
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import linear

WHO_PATH = "who_traffic_death_rate.csv"

# Load data
df = pd.read_csv(WHO_PATH, low_memory=False)

# Only important columns
df = df[['SpatialDimValueCode', 'Location', 'Value']]
df = df.rename(columns={
    'SpatialDimValueCode': 'iso_a3',
    'Location': 'country',
    'Value': 'rate_per_100k'
})

# Natural Earth shapefile
world = gpd.read_file("ne_10m_admin_0_countries.shp")
world = world.rename(columns={"ADM0_A3": "iso_a3"})

world["iso_a3"] = world["iso_a3"].str.upper().str.strip()
df["iso_a3"] = df["iso_a3"].str.upper().str.strip()

# Merge data
world = world.merge(df, on="iso_a3", how="left")

# Folium map
m = folium.Map(tiles='cartodbpositron', zoom_start=2)

# Colormap
colormap = linear.YlOrRd_09.scale(
    world["rate_per_100k"].min(),
    world["rate_per_100k"].max()
)
colormap.caption = 'Road Traffic Death Rate per 100k population (2021)'

# GeoJson
folium.GeoJson(
    world.to_json(),
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['rate_per_100k'])
        if feature['properties'].get('rate_per_100k') is not None else 'lightgray',
        'color': 'black',
        'weight': 0.2,
        'fillOpacity': 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME', 'rate_per_100k'],
        aliases=['Country', 'Deaths per 100k'],
        localize=True
    )
).add_to(m)

colormap.add_to(m)

# Custom legend for no data
no_data_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 150px;
    height: 30px;
    background-color: white;
    border: 1px solid black;
    z-index: 9999;
    font-size:14px;
    padding: 5px;">
    <span style="background-color: lightgray;
                 width: 12px;
                 height: 12px;
                 display: inline-block;
                 border: 1px solid black;
                 margin-right: 6px;">
    </span>
    No data
</div>
"""
m.get_root().html.add_child(folium.Element(no_data_html))

# Title
title_html = '''
<h3 align="center" style="font-size:20px">
Road Traffic Death Rate per 100,000 population (2021)
</h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

m.save('choropleth_who_death_rate.html')
print("Saved choropleth_who_death_rate.html")

Saved choropleth_who_death_rate.html
